# OpenEarhMap Semantinc Segmentation

This demo code demonstrates training and testing of UNet-EfficientNet-B4 for the OpenEarthMap dataset (https://open-earth-map.org/). This demo code is based on the work from the "segmentation_models.pytorch" repository by qubvel, available at: https://github.com/qubvel/segmentation_models.pytorch. We extend our sincere appreciation to the original author for their invaluable contributions to the field of semantic segmentation and for providing this open-source implementation.

---

### Requirements

In [222]:
%pip install rasterio 
%pip install pretrainedmodels 
%pip install efficientnet_pytorch 
%pip install timm 



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import
---

In [223]:
import sys #library ที่จัดเตรียมฟังก์ชันและตัวแปรที่ใช้เพื่อจัดการกับส่วนต่างๆของ Python Runtime Environment
sys.path.append(r'C:\Users\SE\Desktop\yesib\OEM_230725\SPD.ipynb')  # Correct path to the directory containing source.py
import os #นำเข้า module OS มาในโค้ดภาษา Python
import time #คำสั่งต่างๆ มากมายที่เกี่ยวกับเวลา
import numpy as np # library ที่ใช้ในการคำนวนทางคณิตศาสตร์ในภาษา Python
import torch #library for use Machine Learning
import torch.nn as nn #Neural Network คือ Layer ที่ใช้ในการคำนวณค่า
from torch.utils.data import DataLoader #คือ Dataset object และลิสต์ของจำนวนข้อมูลที่ต้องการจะแบ่ง
import source  # Ensure 'source.py' is in the 'final_finish' directory
import segmentation_models_pytorch as smp #
import glob
import torchvision.transforms.functional as TF
import math
import cv2
from PIL import Image
import warnings
from pathlib import Path


warnings.filterwarnings("ignore")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


### Define main parameters

In [224]:
OEM_ROOT = r"C:\Users\SE\Desktop\yesib\OEM_230725\data\OpenEarthMap_Demo"
OEM_DATA_DIR = os.path.join(OEM_ROOT, 'train_val')
TEST_DIR = OEM_ROOT+'test/'
TRAIN_LIST = os.path.join(OEM_ROOT, "train.txt")
VAL_LIST = os.path.join(OEM_ROOT, "val.txt")
WEIGHT_DIR = r"C:\Users\SE\Desktop\yesib\OEM_230725\weight" # path to save weights
OUT_DIR = r"C:\Users\SE\Desktop\yesib\OEM_230725\result" # path to save prediction images
os.makedirs(WEIGHT_DIR, exist_ok=True)
test_large = OEM_ROOT+'/helloworld.tif'

seed = 0
learning_rate = 0.0001
batch_size = 4
n_epochs = 3
classes = [1, 2, 3, 4, 5, 6, 7, 8]
n_classes = len(classes)+1
classes_wt = np.ones([n_classes], dtype=np.float32)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = "cuda" if torch.cuda.is_available() else "cpu"


print("Number of epochs   :", n_epochs)
print("Number of classes  :", n_classes)
print("Batch size         :", batch_size)
print("Device             :", device)

Number of epochs   : 3
Number of classes  : 9
Batch size         : 4
Device             : cuda


### Prepare training and validation file lists

In this demo for Google Colab, we use only two regions, i.e., Tokyo and Kyoto for training. To train with the full set, please download the OpenEarthMap dataset from https://zenodo.org/record/7223446. Note for xBD data preparation is available at https://github.com/bao18/open_earth_map.

In [225]:
img_pths = [f for f in Path(OEM_DATA_DIR).rglob("*.png") if "labels" in str(f)]
train_pths = [str(f) for f in img_pths if f.name in np.loadtxt(TRAIN_LIST, dtype=str)]
val_pths = [str(f) for f in img_pths if f.name in np.loadtxt(VAL_LIST, dtype=str)]

print("Total samples      :", len(img_pths))
print("Training samples   :", len(train_pths))
print("Validation samples :", len(val_pths))

Total samples      : 10001
Training samples   : 9
Validation samples : 1


### Define training and validation dataloaders

In [226]:
trainset = source.dataset.Dataset(train_pths, classes=classes, size=1024, train=True)
validset = source.dataset.Dataset(val_pths, classes=classes, train=False)

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=0)


### Setup network

In [227]:
import torch
print(torch.__version__)  # Check the version of PyTorch
print(torch.cuda.is_available())  # Should return True if CUDA is available



network = smp.Unet(
    classes=n_classes,
    activation=None,
    encoder_weights="imagenet",
    encoder_name="efficientnet-b4",
    decoder_attention_type="scse",
)

# count parameters
params = 0
for p in network.parameters():
    if p.requires_grad:
        params += p.numel()

criterion = source.losses.CEWithLogitsLoss(weights=classes_wt)
criterion_name = 'CE'
metric = source.metrics.IoU2()
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
network_fout = f"{network.name}_s{seed}_{criterion.name}"
OUT_DIR += network_fout # path to save prediction images
os.makedirs(OUT_DIR, exist_ok=True)

print("Model output name  :", network_fout)
print("Number of parameters: ", params)

if torch.cuda.device_count() > 1:
    print("Number of GPUs :", torch.cuda.device_count())
    network = torch.nn.DataParallel(network)
    optimizer = torch.optim.Adam(
        [dict(params=network.module.parameters(), lr=learning_rate)]
    )

2.6.0+cu118
True
https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth
Model output name  : u-efficientnet-b4_s0_CELoss
Number of parameters:  20304278


### Visualization functions

In [228]:
class_rgb = {
    "Bareland": [128, 0, 0],
    "Grass": [0, 255, 36],
    "Pavement": [148, 148, 148],
    "Road": [255, 255, 255],
    "Tree": [34, 97, 38],
    "Water": [0, 69, 255],
    "Cropland": [75, 181, 73],
    "buildings": [222, 31, 7],
}

class_gray = {
    "Bareland": 1,
    "Grass": 2,
    "Pavement": 3,
    "Road": 4,
    "Tree": 5,
    "Water": 6,
    "Cropland": 7,
    "buildings": 8,
}

def label2rgb(a):
    """
    a: labels (HxW)
    """
    out = np.zeros(shape=a.shape + (3,), dtype="uint8")
    for k, v in class_gray.items():
        out[a == v, 0] = class_rgb[k][0]
        out[a == v, 1] = class_rgb[k][1]
        out[a == v, 2] = class_rgb[k][2]
    return out

### Training

In [229]:
### Training
start = time.time()

max_score = 0
train_hist = []
valid_hist = []
best_model_path = os.path.join(WEIGHT_DIR, f"{network_fout}.pth")

# Load best model if it exists before training
if os.path.exists(best_model_path):
    network.load_state_dict(torch.load(best_model_path))
    print(f"🔄 Resuming training from {best_model_path}")

for epoch in range(n_epochs):
    print(f"\n🚀 Epoch: {epoch + 1}")

    logs_train, train_iou_per_class = source.runner.train_epoch(
        model=network,
        optimizer=optimizer,
        criterion=criterion,
        metric=metric,
        dataloader=train_loader,
        device=device,
    )

    logs_valid, valid_iou_per_class = source.runner.valid_epoch(
        model=network,
        criterion=criterion,
        metric=metric,
        dataloader=valid_loader,
        device=device,
    )

    train_hist.append(logs_train)
    valid_hist.append(logs_valid)

    score = logs_valid["mIoU"]

    if max_score < score:
        max_score = score
        torch.save(network.state_dict(), best_model_path)
        print("✅ Best Model saved!")

    # Print per-class IoU for train and validation
    print("\n📌 **IoU per class (Train & Valid) (as %):**")
    for i, (train_iou, valid_iou) in enumerate(zip(train_iou_per_class, valid_iou_per_class), start=0):
        print(f" - Class {i}: Train IoU: {train_iou * 100:.2f}%, Valid IoU: {valid_iou * 100:.2f}%")

    # Find the best-performing class for validation
    best_valid_class = np.nanargmax(valid_iou_per_class) + 1
    best_valid_class_iou = valid_iou_per_class[best_valid_class - 1] * 100
    print(f"\n🏆 **Max IoU Score (Validation): Class {best_valid_class} with {best_valid_class_iou:.2f}%**")

    # Find the best-performing class for training
    best_train_class = np.nanargmax(train_iou_per_class) + 1
    best_train_class_iou = train_iou_per_class[best_train_class - 1] * 100
    print(f"🏆 **Max IoU Score (Train): Class {best_train_class} with {best_train_class_iou:.2f}%**")

    # Print epoch summary
    print(f"\n📌 **Epoch {epoch + 1} Summary**")
    print(f"🏆 Max Train mIoU so far: {logs_train['mIoU'] * 100:.2f}%")
    print(f"🏆 Max Valid mIoU so far: {logs_valid['mIoU'] * 100:.2f}%")

end = time.time()
print("\n⏳ Processing time:", end - start)

# Load best model if needed
if os.path.exists(best_model_path):
    network.load_state_dict(torch.load(best_model_path))
    print(f"🔄 Best Model loaded from {best_model_path}")


🔄 Resuming training from C:\Users\SE\Desktop\yesib\OEM_230725\weight\u-efficientnet-b4_s0_CELoss.pth

🚀 Epoch: 1


Valid: 100%|██████████| 1/1 [00:00<00:00,  1.92it/s, CELoss=2.33, mIoU=2.61%]


✅ Best Model saved!

📌 **IoU per class (Train & Valid) (as %):**
 - Class 0: Train IoU: 0.00%, Valid IoU: 0.00%
 - Class 1: Train IoU: 2.04%, Valid IoU: 7.76%
 - Class 2: Train IoU: 2.67%, Valid IoU: 2.70%
 - Class 3: Train IoU: 0.89%, Valid IoU: 1.38%
 - Class 4: Train IoU: 1.08%, Valid IoU: 4.29%
 - Class 5: Train IoU: 0.11%, Valid IoU: 0.04%
 - Class 6: Train IoU: 0.12%, Valid IoU: 0.04%
 - Class 7: Train IoU: 0.76%, Valid IoU: 0.97%
 - Class 8: Train IoU: 0.63%, Valid IoU: 6.28%

🏆 **Max IoU Score (Validation): Class 2 with 7.76%**
🏆 **Max IoU Score (Train): Class 3 with 2.67%**

📌 **Epoch 1 Summary**
🏆 Max Train mIoU so far: 2.90%
🏆 Max Valid mIoU so far: 2.61%

🚀 Epoch: 2


Valid: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s, CELoss=2.34, mIoU=2.96%]


✅ Best Model saved!

📌 **IoU per class (Train & Valid) (as %):**
 - Class 0: Train IoU: 0.00%, Valid IoU: 0.00%
 - Class 1: Train IoU: 2.97%, Valid IoU: 7.36%
 - Class 2: Train IoU: 3.13%, Valid IoU: 2.69%
 - Class 3: Train IoU: 0.74%, Valid IoU: 1.40%
 - Class 4: Train IoU: 0.99%, Valid IoU: 4.56%
 - Class 5: Train IoU: 0.14%, Valid IoU: 0.06%
 - Class 6: Train IoU: 0.18%, Valid IoU: 0.03%
 - Class 7: Train IoU: 0.86%, Valid IoU: 1.04%
 - Class 8: Train IoU: 0.64%, Valid IoU: 9.50%

🏆 **Max IoU Score (Validation): Class 9 with 9.50%**
🏆 **Max IoU Score (Train): Class 3 with 3.13%**

📌 **Epoch 2 Summary**
🏆 Max Train mIoU so far: 3.12%
🏆 Max Valid mIoU so far: 2.96%

🚀 Epoch: 3


Valid: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s, CELoss=2.31, mIoU=2.98%]


✅ Best Model saved!

📌 **IoU per class (Train & Valid) (as %):**
 - Class 0: Train IoU: 0.00%, Valid IoU: 0.00%
 - Class 1: Train IoU: 2.01%, Valid IoU: 6.82%
 - Class 2: Train IoU: 3.04%, Valid IoU: 3.09%
 - Class 3: Train IoU: 0.83%, Valid IoU: 1.65%
 - Class 4: Train IoU: 1.11%, Valid IoU: 4.52%
 - Class 5: Train IoU: 0.24%, Valid IoU: 0.07%
 - Class 6: Train IoU: 0.19%, Valid IoU: 0.03%
 - Class 7: Train IoU: 0.87%, Valid IoU: 1.20%
 - Class 8: Train IoU: 0.78%, Valid IoU: 9.41%

🏆 **Max IoU Score (Validation): Class 9 with 9.41%**
🏆 **Max IoU Score (Train): Class 3 with 3.04%**

📌 **Epoch 3 Summary**
🏆 Max Train mIoU so far: 3.18%
🏆 Max Valid mIoU so far: 2.98%

⏳ Processing time: 85.49411153793335
🔄 Best Model loaded from C:\Users\SE\Desktop\yesib\OEM_230725\weight\u-efficientnet-b4_s0_CELoss.pth


### Testing


In [230]:
start = time.time()

# load network
network.load_state_dict(torch.load(os.path.join(WEIGHT_DIR, f"{network_fout}.pth")))
network.to(device).eval()

test_pths = glob.glob(TEST_DIR+"/*.tif")
#testset = source.dataset.Dataset(test_pths, classes=classes, train=False)

for fn_img in test_pths:
  img = source.dataset.load_multiband(fn_img)
  h, w = img.shape[:2]
  power = math.ceil(np.log2(h) / np.log2(2))
  shape = (2 ** power, 2 ** power)
  img = cv2.resize(img, shape)

  # test time augmentation
  imgs = []
  imgs.append(img.copy())
  imgs.append(img[:, ::-1, :].copy())
  imgs.append(img[::-1, :, :].copy())
  imgs.append(img[::-1, ::-1, :].copy())

  input = torch.cat([TF.to_tensor(x).unsqueeze(0) for x in imgs], dim=0).float().to(device)

  pred = []
  with torch.no_grad():
      msk = network(input)
      msk = torch.softmax(msk[:, :, ...], dim=1)
      msk = msk.cpu().numpy()
      pred = (msk[0, :, :, :] + msk[1, :, :, ::-1] + msk[2, :, ::-1, :] + msk[3, :, ::-1, ::-1])/4
  pred = pred.argmax(axis=0).astype("uint8")
  size = pred.shape[0:]
  y_pr = cv2.resize(pred, (w, h), interpolation=cv2.INTER_NEAREST)

  # save image as png
  filename = os.path.splitext(os.path.basename(fn_img))[0]
  y_pr_rgb = label2rgb(y_pr)
  Image.fromarray(y_pr_rgb).save(os.path.join(OUT_DIR, filename+'test.png'))

end = time.time()
print('Processing time:',end - start)

Processing time: 0.11789298057556152


### Testing a model for a large Geotiff image

A sample image is provided by the Geospatial Information Authority of Japan at https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg


In [231]:
start = time.time()

# load network
network.load_state_dict(torch.load(os.path.join(WEIGHT_DIR, f"{network_fout}.pth")))
network.to(device).eval()

# process large Geotiff image
img0 = source.dataset.load_multiband(test_large)

# get crs and transform
crs, trans = source.dataset.get_crs(test_large)

if img0.shape[2] > 3:
    img0 = img0[:,:,[0,1,2]]
height = img0.shape[0]
width = img0.shape[1]
band = img0.shape[2]

patch_size = 1024
stride = 256
C = int(np.ceil( (width - patch_size) / stride ) + 1)
R = int(np.ceil( (height - patch_size) / stride ) + 1)

# weight matrix B for avoiding boundaries of patches
if patch_size > stride:
    w = patch_size
    s1 = stride
    s2 = w - s1
    d = 1/(1+s2)
    B1 = np.ones((w,w))
    B1[:,s1::] = np.dot(np.ones((w,1)),(-np.arange(1,s2+1)*d+1).reshape(1,s2))
    B2 = np.flip(B1)
    B3 = B1.T
    B4 = np.flip(B3)
    B = B1*B2*B3*B4
else:
    B = np.ones((w,w))

img1 = np.zeros((patch_size+stride*(R-1), patch_size+stride*(C-1),3))
img1[0:height,0:width,:] = img0.copy()

pred_all = np.zeros((9,patch_size+stride*(R-1), patch_size+stride*(C-1)))
weight = np.zeros((patch_size+stride*(R-1), patch_size+stride*(C-1)))

for r in range(R):
    for c in range(C):
        img = img1[r*stride:r*stride+patch_size,c*stride:c*stride+patch_size,:].copy().astype(np.float32)/255
        imgs = []
        imgs.append(img.copy())
        imgs.append(img[:, ::-1, :].copy())
        imgs.append(img[::-1, :, :].copy())
        imgs.append(img[::-1, ::-1, :].copy())

        input = torch.cat([TF.to_tensor(x).unsqueeze(0) for x in imgs], dim=0).float().to(device)

        pred = []
        with torch.no_grad():
            msk = network(input)
            msk = torch.softmax(msk[:, :, ...], dim=1)
            msk = msk.cpu().numpy()

            pred = (msk[0, :, :, :] + msk[1, :, :, ::-1] + msk[2, :, ::-1, :] + msk[3, :, ::-1, ::-1])/4

        pred_all[:,r*stride:r*stride+patch_size,c*stride:c*stride+patch_size] += pred.copy()*B
        weight[r*stride:r*stride+patch_size,c*stride:c*stride+patch_size] += B

for b in range(9):
    pred_all[b,:,:] = pred_all[b,:,:]/weight
    if b == 0:
        pred_all[b,:,:] = 0

pred_all = pred_all.argmax(axis=0).astype("uint8")

filename = os.path.splitext(os.path.basename(test_large))[0]
pr_rgb = label2rgb(pred_all)
Image.fromarray(pr_rgb[0:height,0:width,:]).save(os.path.join(OUT_DIR, filename+'_pr.png'))

# save geotiff
pr_rgb = np.transpose(pr_rgb[0:height,0:width,:], (2,0,1))
source.dataset.save_img(os.path.join(OUT_DIR, filename+'_pr.tif'),pr_rgb,crs,trans)

end = time.time()
print('Processing time:',end - start)

KeyboardInterrupt: 